In [1]:
import pandas as pd

In [93]:
transaction = pd.read_csv('TRANSACTION_TAKEHOME.csv')
products = pd.read_csv('PRODUCTS_TAKEHOME.csv')
user = pd.read_csv('USER_TAKEHOME.csv')

### 1. Data Exploration and Data Cleanup 

#### 1.1 Transaction Dataset

In [56]:
transaction.head()

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,


In [57]:
# Count missing values in each column 
print(transaction.isnull().sum())

RECEIPT_ID           0
PURCHASE_DATE        0
SCAN_DATE            0
STORE_NAME           0
USER_ID              0
BARCODE           5762
FINAL_QUANTITY       0
FINAL_SALE           0
dtype: int64


In [58]:
# Check data types of each column
print(transaction.dtypes)

RECEIPT_ID         object
PURCHASE_DATE      object
SCAN_DATE          object
STORE_NAME         object
USER_ID            object
BARCODE           float64
FINAL_QUANTITY     object
FINAL_SALE         object
dtype: object


In [59]:
# Convert columns to appropriate data types
transaction['PURCHASE_DATE'] = pd.to_datetime(transaction['PURCHASE_DATE'])
transaction['SCAN_DATE'] = pd.to_datetime(transaction['SCAN_DATE'])

In [60]:
transaction['FINAL_QUANTITY'] = pd.to_numeric(transaction['FINAL_QUANTITY'], errors='coerce')  # Convert to float

In [61]:
transaction['FINAL_QUANTITY'] = transaction['FINAL_QUANTITY'].replace('zero', 0).fillna(0).astype(int) # Convert to integer

In [62]:
transaction['FINAL_QUANTITY'] 

0        1
1        0
2        1
3        0
4        1
        ..
49995    2
49996    1
49997    1
49998    1
49999    1
Name: FINAL_QUANTITY, Length: 50000, dtype: int32

In [65]:
# Convert FINAL_SALE to numeric, coercing errors (e.g., None to NaN)
transaction['FINAL_SALE'] = pd.to_numeric(transaction['FINAL_SALE'], errors='coerce')

In [66]:
#data types after cleanup
transaction.dtypes

RECEIPT_ID                     object
PURCHASE_DATE          datetime64[ns]
SCAN_DATE         datetime64[ns, UTC]
STORE_NAME                     object
USER_ID                        object
BARCODE                       float64
FINAL_QUANTITY                  int32
FINAL_SALE                    float64
dtype: object

In [69]:
# Convert PURCHASE_DATE to timezone-aware
transaction['PURCHASE_DATE'] = pd.to_datetime(transaction['PURCHASE_DATE']).dt.tz_localize('UTC')
# Add a column to flag rows where SCAN_DATE < PURCHASE_DATE
transaction['DATE_DISCREPANCY'] = transaction['SCAN_DATE'] < transaction['PURCHASE_DATE']

In [76]:
#filter rows where scan date is before purchase date
transaction[transaction['DATE_DISCREPANCY'] == True]

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,DATE_DISCREPANCY
51,008c1dcc-0f96-4b04-98c8-2a2bb63ef89d,2024-07-21 00:00:00+00:00,2024-07-20 19:54:23.133000+00:00,WALMART,5dc24cdb682fcf1229d04bd6,6.811312e+11,0,3.18,True
455,04a320ed-2903-45e5-8fd7-6eaf08daef32,2024-06-29 00:00:00+00:00,2024-06-28 11:03:31.783000+00:00,DOLLAR GENERAL STORE,62855f67708670299a658035,4.900002e+10,0,6.00,True
494,05023b3d-5f83-47a7-a17c-8e8521d0bc94,2024-09-08 00:00:00+00:00,2024-09-07 22:22:29.903000+00:00,SHOP RITE,666a43c77c0469953bfd9ae0,6.414404e+10,2,NaN,True
675,06ce3da3-a588-4c37-93b4-0b6d11e42704,2024-06-22 00:00:00+00:00,2024-06-21 12:34:15.665000+00:00,BIG LOTS,646f6ffb7a342372c858487e,3.111117e+11,0,4.05,True
870,08d0e78f-3e63-40a3-8eb0-73fdf76da52c,2024-06-22 00:00:00+00:00,2024-06-21 20:50:01.298000+00:00,DOLLAR GENERAL STORE,664cafb6e04f743a096a837e,7.680828e+10,2,NaN,True
...,...,...,...,...,...,...,...,...,...
46034,08d0e78f-3e63-40a3-8eb0-73fdf76da52c,2024-06-22 00:00:00+00:00,2024-06-21 20:50:01.298000+00:00,DOLLAR GENERAL STORE,664cafb6e04f743a096a837e,7.680828e+10,2,3.50,True
46539,718aa730-b62f-4e18-8dba-1d7105dac341,2024-09-05 00:00:00+00:00,2024-09-04 20:14:00.374000+00:00,WALMART,5e0f561efa890112094202ad,7.339001e+10,1,3.56,True
46941,af2b818f-4a92-4e98-958c-65f2ce0b271d,2024-06-15 00:00:00+00:00,2024-06-14 10:57:23.892000+00:00,DOLLAR GENERAL STORE,64de6465516348066e7c5690,8.175130e+11,1,3.83,True
47653,72bb7b71-d958-4a46-ae62-43abdeb0e693,2024-06-15 00:00:00+00:00,2024-06-14 19:55:56.672000+00:00,WALMART,649726ea127ddb5d7f0004dc,3.485611e+10,1,2.05,True


The Code shows the rows with date discrepancies. There are 9 rows where scan date is before purchase date, meaning there is something wrong with these records. We should take a look into the record and see where it went wrong to improve our data recordings later on.

#### 1.2 Users Dataset

In [94]:
user.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54.000 Z,2000-08-11 00:00:00.000 Z,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55.000 Z,2001-09-24 04:00:00.000 Z,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18.000 Z,1994-10-28 00:00:00.000 Z,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22.000 Z,NaN,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50.000 Z,1972-03-19 00:00:00.000 Z,PA,en,female


In [95]:
# show data type for each column before any clean up
user.dtypes 

ID              object
CREATED_DATE    object
BIRTH_DATE      object
STATE           object
LANGUAGE        object
GENDER          object
dtype: object

In [96]:
# Convert CREATED_DATE and BIRTH_DATE to datetime
user['CREATED_DATE'] = pd.to_datetime(user['CREATED_DATE'], errors='coerce')
user['BIRTH_DATE'] = pd.to_datetime(user['BIRTH_DATE'], errors='coerce')

In [97]:
user.head()

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54+00:00,2000-08-11 00:00:00+00:00,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55+00:00,2001-09-24 04:00:00+00:00,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18+00:00,1994-10-28 00:00:00+00:00,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22+00:00,NaT,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50+00:00,1972-03-19 00:00:00+00:00,PA,en,female


In [98]:
# Check missing values
user.isnull().sum()

ID                  0
CREATED_DATE        0
BIRTH_DATE       3675
STATE            4812
LANGUAGE        30508
GENDER           5892
dtype: int64

In [99]:
# Display unique states
unique_states = user['STATE'].unique()
print("Unique states in the dataset:", unique_states)

Unique states in the dataset: ['CA' 'PA' 'FL' 'NC' 'NY' 'IN' nan 'OH' 'TX' 'NM' 'PR' 'CO' 'AZ' 'RI' 'MO'
 'NJ' 'MA' 'TN' 'LA' 'NH' 'WI' 'IA' 'GA' 'VA' 'DC' 'KY' 'SC' 'MN' 'WV'
 'DE' 'MI' 'IL' 'MS' 'WA' 'KS' 'CT' 'OR' 'UT' 'MD' 'OK' 'NE' 'NV' 'AL'
 'AK' 'AR' 'HI' 'ME' 'ND' 'ID' 'WY' 'MT' 'SD' 'VT']


In [100]:
# Display unique languages
unique_languages = user['LANGUAGE'].unique()
print("Unique languages in the dataset:", unique_languages)

Unique languages in the dataset: ['es-419' 'en' nan]


In [91]:
duplicates = user[user.duplicated(subset=['ID'])]
print(f"Number of duplicate rows: {len(duplicates)}")

Number of duplicate rows: 0


There are no duplicate users in the dataset, so it it good to go

In [104]:
user = user.copy()  
user['GENDER'] = user['GENDER'].fillna('Unknown') #fill in the null values with unknown

In [105]:
user['LANGUAGE'] = user['LANGUAGE'].fillna('Unknown') #fill in the null values with unknown

In [107]:
user['BIRTH_DATE'] = user['BIRTH_DATE'].fillna(pd.Timestamp("1900-01-01")) #Replace missing values with a placeholder

#### 1.3 Products Dataset

In [109]:
products.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,7.964944e+11
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,2.327801e+10
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,4.618178e+11
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,3.500047e+10
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,8.068109e+11


In [125]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845552 entries, 0 to 845551
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   CATEGORY_1    845441 non-null  object 
 1   CATEGORY_2    844128 non-null  object 
 2   CATEGORY_3    784986 non-null  object 
 3   CATEGORY_4    67459 non-null   object 
 4   MANUFACTURER  845552 non-null  object 
 5   BRAND         845552 non-null  object 
 6   BARCODE       841527 non-null  float64
dtypes: float64(1), object(6)
memory usage: 45.2+ MB


In [117]:
# Check missing values
products.isnull().sum()

CATEGORY_1         111
CATEGORY_2        1424
CATEGORY_3       60566
CATEGORY_4      778093
MANUFACTURER         0
BRAND                0
BARCODE           4025
dtype: int64

Something to notice here is number of missing values for barcode. This column is essential to recognize which product it is referring to. Without this data, I think it will be hard to make insights from those records. But I will still keep those records for now in case it will influence our analysis later on.

In [110]:
products['MANUFACTURER'] = products['MANUFACTURER'].replace("PLACEHOLDER MANUFACTURER", "Unknown")

In [111]:
products['MANUFACTURER'] = products['MANUFACTURER'].fillna('Unknown') #fill in the null values with unknown

In [112]:
products['BRAND'] = products['BRAND'].fillna('Unknown') #fill in the null values with unknown

In [126]:
# Drop duplicates based on all columns
products_cleaned = products.drop_duplicates()

In [127]:
products_cleaned 

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,Unknown,Unknown,7.964944e+11
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,Unknown,Unknown,2.327801e+10
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,Unknown,ELECSOP,4.618178e+11
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,3.500047e+10
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,8.068109e+11
...,...,...,...,...,...,...,...
845547,Health & Wellness,Topical Muscle & Joint Relief Treatments,Braces & Wraps,NaN,Unknown,Unknown,7.223016e+11
845548,Snacks,Cookies,NaN,NaN,"TREEHOUSE FOODS, INC.",LOFTHOUSE,4.182082e+10
845549,Snacks,Candy,Confection Candy,NaN,HARIBO GMBH & CO KG,HARIBO,1.001672e+11
845550,Snacks,Nuts & Seeds,Hazelnuts,NaN,DOUBLE-COLA CO,JUMBO,7.539076e+10


In [132]:
# Exclude null BARCODE values
non_null_barcodes = products_cleaned [products_cleaned ['BARCODE'].notnull()]

# Find duplicate BARCODE entries
duplicate_barcodes = non_null_barcodes[non_null_barcodes.duplicated(subset=['BARCODE'], keep=False)]

# Display duplicates
print("Duplicate BARCODE entries (excluding null values):\n", duplicate_barcodes)

Duplicate BARCODE entries (excluding null values):
                CATEGORY_1    CATEGORY_2                   CATEGORY_3  \
162     Health & Wellness  Hair Removal          Shaving Gel & Cream   
28421   Health & Wellness     Hair Care                   Hair Color   
36017              Snacks         Candy           Candy Variety Pack   
37152              Snacks         Candy             Confection Candy   
56987              Snacks  Nuts & Seeds                      Almonds   
96438              Snacks      Crackers               Other Crackers   
108325             Snacks         Candy                        Mints   
114131             Snacks         Candy              Chocolate Candy   
123194  Health & Wellness     Skin Care       Lip Balms & Treatments   
127335             Snacks  Nuts & Seeds                   Pistachios   
132547             Snacks       Cookies                          NaN   
137250             Snacks  Nuts & Seeds                      Peanuts   
139121      

From this result, we see that there are still some duplicate barcodes in the dataset. These duplicate barcodes have at least one category name that is different. To resolve this, I group the data by barcode, and join the category if it is not the same. This will ensure we only have one record for each barcode. The code is performed below.

In [135]:
# Group by BARCODE and aggregate the category columns
merged_products = (
    products.groupby('BARCODE', dropna=False)
    .agg({
        'CATEGORY_1': lambda x: ', '.join(x.dropna().unique()),
        'CATEGORY_2': lambda x: ', '.join(x.dropna().unique()),
        'CATEGORY_3': lambda x: ', '.join(x.dropna().unique()),
        'CATEGORY_4': lambda x: ', '.join(x.dropna().unique()),
        'MANUFACTURER': lambda x: ', '.join(x.dropna().unique()),
        'BRAND': lambda x: ', '.join(x.dropna().unique()),
    })
    .reset_index()
)

# Display the consolidated dataset
print(merged_products)

             BARCODE                                         CATEGORY_1  \
0       1.850000e+02                                          Beverages   
1       3.582000e+03                                  Health & Wellness   
2       4.091000e+03                                             Snacks   
3       5.579000e+03                                  Health & Wellness   
4       5.777000e+03                                          Beverages   
...              ...                                                ...   
841338  5.010725e+13                                  Health & Wellness   
841339  5.411446e+13                                             Snacks   
841340  6.291101e+13                                             Snacks   
841341  6.291108e+13                                             Snacks   
841342           NaN  Health & Wellness, Snacks, Apparel & Accessori...   

                                               CATEGORY_2  \
0                                  Car

In [134]:
# Exclude null BARCODE values
non_null_barcodes = merged_products [merged_products ['BARCODE'].notnull()]

# Find duplicate BARCODE entries
duplicate_barcodes = non_null_barcodes[non_null_barcodes.duplicated(subset=['BARCODE'], keep=False)]

# Display duplicates
print("Duplicate BARCODE entries (excluding null values):\n", duplicate_barcodes)

Duplicate BARCODE entries (excluding null values):
 Empty DataFrame
Columns: [BARCODE, CATEGORY_1, CATEGORY_2, CATEGORY_3, CATEGORY_4, MANUFACTURER, BRAND]
Index: []


As a result, we can see there are no duplicate barcodes excluding the null barcodes.

#### 2. Summarize the Questions

##### 1. Are there any data quality issues present?

To summarize the data issues, there are missing values and duplicates across the datasets, with variations in their nature and impact.

For the transaction dataset, there are missing values in FINAL_SALE and BARCODE. This could lead to incomplete information about the total receipt amounts and product details. A more significant issue is the date discrepancies observed in some cases, where the SCAN_DATE occurs before the PURCHASE_DATE, which is logically inconsistent.

For the user dataset, missing values are present in BIRTH_DATE, GENDER, and STATE. While these gaps may not critically affect the analysis, having more complete information would enhance our understanding of the customers. However, addressing these gaps is likely not the top priority at this stage.

Finally, for the products dataset, the primary concern lies with the BARCODE. There are some missing values (approximately 0.4%), and there are duplicate records for the same BARCODE. From my understanding, each product should correspond to a unique BARCODE, making it the primary key of the table. These duplicates and missing values could cause inconsistencies in identifying products accurately.

##### 2. Are there any fields that are challenging to understand?

I was a bit confused about final quantity and final sale in the transaction dataset because there are zeros in final quantity where final sale has a number and some final quantities with number but final sale has no number. Also, I think the purpose and hierarchy of CATEGORY_1 through CATEGORY_4 might need some more explanations.